## imports

In [25]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
import warnings
import os

# Suppress warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)

## load raw data

In [26]:
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")

train.set_index('entity_id', inplace=True)
test.set_index('entity_id', inplace=True)

print(train.shape, test.shape)

(429, 11) (49, 9)


## Cleaning Function

In [27]:
def initial_clean_and_engineer(df: pd.DataFrame, is_training: bool) -> pd.DataFrame:
    
    cols_to_clean = [
        'revenue', 'overall_score',
        'environmental_score', 'social_score', 'governance_score'
    ]

    if is_training:
        cols_to_clean.extend(['target_scope_1', 'target_scope_2'])

    # Convert to numeric
    for col in cols_to_clean:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # Impute score columns with median
    score_cols = ['overall_score', 'environmental_score', 'social_score', 'governance_score']
    for col in score_cols:
        df[col] = df[col].fillna(df[col].median())

    # Drop rows missing essential columns
    df.dropna(subset=cols_to_clean, inplace=True)

    # Log transforms
    df['log_revenue'] = np.log1p(df['revenue'])
    df.drop(columns=['revenue'], inplace=True)

    if is_training:
        df['log_scope_1'] = np.log1p(df['target_scope_1'])
        df['log_scope_2'] = np.log1p(df['target_scope_2'])
        df.drop(columns=['target_scope_1', 'target_scope_2'], inplace=True)

    # Drop unused text fields
    df.drop(columns=['region_name', 'country_name'], inplace=True, errors='ignore')

    return df

## Apply Cleaning & Split Features/Targets

In [28]:
train_clean = initial_clean_and_engineer(train.copy(), is_training=True)
test_clean = initial_clean_and_engineer(test.copy(), is_training=False)

TARGETS = ['log_scope_1', 'log_scope_2']
X_train = train_clean.drop(columns=TARGETS)
Y_train = train_clean[TARGETS]
X_test = test_clean.copy()

print("Initial cleaning complete.")
print("Train shape:", X_train.shape, "Test shape:", X_test.shape)

Initial cleaning complete.
Train shape: (429, 7) Test shape: (49, 7)


## Merge Prep

In [29]:
train_features = X_train.copy()
test_features  = X_test.copy()

print("Starting external merges...")

Starting external merges...


## External Feature: Sector Revenue Distribution

In [30]:
try:
    sect = pd.read_csv("../data/revenue_distribution_by_sector.csv")
    level_1_sect = (
        sect.pivot_table(
            values='revenue_pct',
            index='entity_id',
            columns='nace_level_1_code',
            aggfunc='sum',
            fill_value=0
        )
        .add_prefix('sect_')
        .add_suffix('_pct')
    )

    train_features = train_features.merge(level_1_sect, left_index=True, right_index=True, how='left')
    test_features = test_features.merge(level_1_sect, left_index=True, right_index=True, how='left')

    sector_cols = train_features.filter(like='sect_').columns
    train_features[sector_cols] = train_features[sector_cols].fillna(0)
    test_features[sector_cols] = test_features[sector_cols].fillna(0)

    print("Sector features merged.")
except FileNotFoundError:
    print("WARNING: Sector file not found.")

Sector features merged.


## External Feature: Environmental Adjustments

In [31]:
try:
    env = pd.read_csv("../data/environmental_activities.csv")
    env_adj = env.groupby('entity_id')['env_score_adjustment'].sum()

    train_features = train_features.merge(env_adj, left_index=True, right_index=True, how='left')
    test_features = test_features.merge(env_adj, left_index=True, right_index=True, how='left')

    train_features['env_score_adjustment'] = train_features['env_score_adjustment'].fillna(0)
    test_features['env_score_adjustment'] = test_features['env_score_adjustment'].fillna(0)

    print("Environmental features merged.")
except FileNotFoundError:
    print("WARNING: Environmental activity file not found.")

Environmental features merged.


## External Features: SDG Aggregation

In [32]:
try:
    sdg = pd.read_csv("../data/sustainable_development_goals.csv")
    sdg = sdg.drop(columns=['sdg_name'])
    sdg = pd.get_dummies(sdg, columns=['sdg_id'])
    sdg_agg = sdg.groupby('entity_id').sum()

    train_features = train_features.merge(sdg_agg, left_index=True, right_index=True, how='left')
    test_features = test_features.merge(sdg_agg, left_index=True, right_index=True, how='left')

    sdg_cols = train_features.filter(like='sdg_id_').columns
    train_features[sdg_cols] = train_features[sdg_cols].fillna(0)
    test_features[sdg_cols] = test_features[sdg_cols].fillna(0)

    print("SDG features merged.")
except FileNotFoundError:
    print("WARNING: SDG file not found.")

SDG features merged.


## final alignment and One-Hot Encoding

In [33]:
categorical_cols = ['region_code', 'country_code']

train_features = pd.get_dummies(train_features, columns=categorical_cols, prefix=categorical_cols)
test_features = pd.get_dummies(test_features, columns=categorical_cols, prefix=categorical_cols)

# Align columns
train_cols = train_features.columns.tolist()
test_features = test_features.reindex(columns=train_cols, fill_value=0)

print("OHE and alignment complete.")

OHE and alignment complete.


## scaling

In [34]:
numeric_cols = train_features.select_dtypes(include=np.number).columns.tolist()

scaler = StandardScaler()
train_features[numeric_cols] = scaler.fit_transform(train_features[numeric_cols])
test_features[numeric_cols] = scaler.transform(test_features[numeric_cols])

print("Scaling complete.")

Scaling complete.


## Variance Threshold

In [35]:
selector = VarianceThreshold(threshold=0.05)
selector.fit(train_features)

selected_columns = train_features.columns[selector.get_support()]

train_features = train_features[selected_columns]
test_features = test_features[selected_columns]

print("Variance selection complete. Features kept:", len(selected_columns))

Variance selection complete. Features kept: 46


## Final output

In [36]:
final_train = train_features.join(Y_train)
final_test = test_features.copy()

final_train.to_pickle("final_train.pkl")
final_test.to_pickle("final_test.pkl")

print("Pipeline complete!")
print("Final train shape:", final_train.shape)
print("Final test shape:", final_test.shape)

Pipeline complete!
Final train shape: (429, 48)
Final test shape: (49, 46)
